In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
BASE_DIR = '/content/drive/MyDrive/cv_project_fall_2022/'
sys.path.append(BASE_DIR)

In [ ]:
import torch
from torch.utils.data import DataLoader


import utils.utils as utils
from models.definitions.transformer_net import TransformerNet

In [ ]:
root = BASE_DIR + "/face_mask/data/widerface"

In [ ]:
import os
from os.path import abspath, expanduser
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import numpy as np


split = "train"

In [ ]:
from matplotlib import pyplot as plt
import cv2
import math

def is_outside_shape(shape, center, width, height, point):
    # center location of face
    cx,cy = center
    
    # candidate mask point
    tx,ty = point
    
    if(shape == 'ellipse'):
        major_axis = width/2
        minor_axis = height/2
        if(height < width):
            major_axis = height/2
            minor_axis = width/2
        return (((tx-cx)*(tx-cx)/(major_axis*major_axis) +((ty-cy)*(ty-cy))/(minor_axis*minor_axis)) > 1)
    
    return True
      
      

  
    
def datagen(img_file_name, annotations, output_image_path, overlay_mask_path, mask_shape):
    
    img = cv2.imread(img_file_name)
    label = np.zeros((img.shape[0],img.shape[1]))
    
    for annotation in annotations:
    
        # from annotation in a single line - using unpacking in Python
        top_left_x, top_left_y, width_of_face, height_of_face = annotation
        cx,cy = top_left_x + int(width_of_face/2) , top_left_y + int(height_of_face/2)
        
        # We create a queue to take account of all candidate mask pixels(neighbours of a mask pixel) 
        # and initialize with the center point of face which we know for sure sould be a mask pixel
        queue = [(cx, cy)]
        
        neighbors = [(0,1),(0,-1),(1,0),(-1,0)]
        
        while(queue):
            tx, ty = queue.pop(0)
            if(tx >= img.shape[0] or ty >= img.shape[1]):
                continue
            
            # Check if pixel is already marked or if it is outside the mask shape
            if(label[ty,tx] or is_outside_shape(mask_shape, (cx,cy), width_of_face, height_of_face, (tx,ty))):
                # No need to fill this value - because outside the bounds of the shape or its already been filled
                continue 
                
            # mark the pixel; we'll multiply the mask by 255 for visualization
            label[ty,tx] = 1 
            
            # push to queue all the neighboring pixels of the candidate pixel (tx,ty)
            for i,j in neighbors:
                x = tx + i
                y = ty + j
                # check for ignoring out of bounds pixels
                if(0<=x<img.shape[1] and 0<=y<img.shape[0]):
                    queue.append((x,y))
    
    # Create a 3 channel image with same height and width as the input image
    label_ = np.zeros((img.shape[0],img.shape[1],3),dtype= np.uint8)
    
    # Fill the 3 channel image with mask
    label_[:,:,0] = label
    label_[:,:,1] = label
    label_[:,:,2] = label
    
    dst = cv2.addWeighted(img[ :, :, ::-1 ],0.5,label_*255,0.7,0)
    dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    
    mode = 0o666

    mask_path = "/".join(output_image_path.split("/")[:-1])
    if( not os.path.exists(mask_path)):
        os.mkdir(mask_path, mode)

    overlay_mask_dir = "/".join(overlay_mask_path.split("/")[:-1])
    if( not os.path.exists(overlay_mask_dir)):
        os.mkdir(overlay_mask_dir, mode)

    

    # Write the highlighted output image(dst) at output_image_path
    mask_img = np.array(label*255).astype('uint8')
    maskImage = cv2.cvtColor(mask_img, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(output_image_path, maskImage)

    #overlay
    cv2.imwrite(overlay_mask_path, dst)
    #print("Mask Name : ", output_image_path)
    
    return output_image_path

In [ ]:
def generate_and_save_mask(img_path, mask_path, overlay_mask_path,bboxes, shape = 'ellipse'):
    datagen(img_path, bboxes, mask_path, overlay_mask_path, shape)



In [ ]:
# Load Wider face annotations file
count = 0
filepath = BASE_DIR + "/face_mask/data/widerface/wider_face_split/wider_face_train_bbx_gt.txt"

img_info: List[Dict[str, Union[str, Dict[str, torch.Tensor]]]] = []

start_count = 18926

with open(filepath) as f:
    lines = f.readlines()
    file_name_line, num_boxes_line, box_annotation_line = True, False, False
    num_boxes, box_counter = 0, 0
    labels = []
    for line in lines:
        line = line.rstrip()
        if file_name_line:
            img_path = os.path.join(root, "WIDER_" + split, "images", line)
            mask_path = os.path.join(root, "WIDER_" + split, "masks", line)
            overlay_mask_path = os.path.join(root, "WIDER_" + split, "overlay_mask", line)
            img_path = abspath(expanduser(img_path))
            file_name_line = False
            num_boxes_line = True
        elif num_boxes_line:
            num_boxes = int(line)
            num_boxes_line = False
            box_annotation_line = True
        elif box_annotation_line:
            box_counter += 1
            line_split = line.split(" ")
            line_values = [int(x) for x in line_split]
            labels.append(line_values)
            if box_counter >= num_boxes:
                box_annotation_line = False
                file_name_line = True
                labels_tensor = torch.tensor(labels)
                # print("Count = ", count)
                # print("Image Name : ", img_path)
                if( not os.path.exists(mask_path)):
                    print("Count = ", count)
                    print("Image Name : ", img_path)
                    generate_and_save_mask(img_path, mask_path, overlay_mask_path, labels_tensor[:, 0:4])
                else :
                    print("Skip : ", count)
                img_info.append(
                    {
                        "img_path": img_path,
                        "mask_path" : mask_path,
                        "overlay_mask_path" : overlay_mask_path,
                        "annotations": {
                            "bbox": labels_tensor[:, 0:4],  # x, y, width, height
                            "blur": labels_tensor[:, 4],
                            "expression": labels_tensor[:, 5],
                            "illumination": labels_tensor[:, 6],
                            "occlusion": labels_tensor[:, 7],
                            "pose": labels_tensor[:, 8],
                            "invalid": labels_tensor[:, 9],
                        },
                    }
                )
                # if(count > 5):
                #     break
                box_counter = 0
                labels.clear()
                count = count + 1
        else:
            raise RuntimeError(f"Error parsing annotation file {filepath}")


Streaming output truncated to the last 5000 lines.
Skip :  7880
Skip :  7881
Skip :  7882
Skip :  7883
Skip :  7884
Skip :  7885
Skip :  7886
Skip :  7887
Skip :  7888
Skip :  7889
Skip :  7890
Skip :  7891
Skip :  7892
Skip :  7893
Skip :  7894
Skip :  7895
Skip :  7896
Skip :  7897
Skip :  7898
Skip :  7899
Skip :  7900
Skip :  7901
Skip :  7902
Skip :  7903
Skip :  7904
Skip :  7905
Skip :  7906
Skip :  7907
Skip :  7908
Skip :  7909
Skip :  7910
Skip :  7911
Skip :  7912
Skip :  7913
Skip :  7914
Skip :  7915
Skip :  7916
Skip :  7917
Skip :  7918
Skip :  7919
Skip :  7920
Skip :  7921
Skip :  7922
Skip :  7923
Skip :  7924
Skip :  7925
Skip :  7926
Skip :  7927
Skip :  7928
Skip :  7929
Skip :  7930
Skip :  7931
Skip :  7932
Skip :  7933
Skip :  7934
Skip :  7935
Skip :  7936
Skip :  7937
Skip :  7938
Skip :  7939
Skip :  7940
Skip :  7941
Skip :  7942
Skip :  7943
Skip :  7944
Skip :  7945
Skip :  7946
Skip :  7947
Skip :  7948
Skip :  7949
Skip :  7950
Skip :  7951
Skip :  7952


In [ ]:
print(count)
img_info[0]

12880


{'img_path': '/content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_849.jpg',
 'mask_path': '/content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/masks/0--Parade/0_Parade_marchingband_1_849.jpg',
 'overlay_mask_path': '/content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/overlay_mask/0--Parade/0_Parade_marchingband_1_849.jpg',
 'annotations': {'bbox': tensor([[449, 330, 122, 149]]),
  'blur': tensor([0]),
  'expression': tensor([0]),
  'illumination': tensor([0]),
  'occlusion': tensor([0]),
  'pose': tensor([0]),
  'invalid': tensor([0])}}

In [ ]:
img_info[0]['img_path'].replace("/images/","/face_masked/")

'/content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/face_masked/0--Parade/0_Parade_marchingband_1_849.jpg'

In [ ]:
path = BASE_DIR + "/face_mask/data/widerface/WIDER_train/masks/0--Parade/0_Parade_marchingband_1_849.jpg"
"/".join(path.split("/")[:-1])

'/content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/masks/0--Parade'

In [ ]:
mode = 0o666
count = 1

# Create mask and overlay results from mask files generated - synthetic dataset for face segmentation
for img in img_info:
    face_masked_img_path = img['img_path'].replace("/images/","/face_masked/")
    face_masked_img_dir = "/".join(face_masked_img_path.split("/")[:-1])
    #print(face_masked_img_path)
    #break
    if( not os.path.exists(face_masked_img_dir)):
        os.mkdir(face_masked_img_dir, mode)
    
    img_path = img['img_path']
    mask_path = img['mask_path']
    result_path = face_masked_img_path
    # Load image, create mask, and draw white circle on mask
    image = cv2.imread(img_path)
    mask = cv2.imread(mask_path) / 255.0
    # Color background white
    result = (image * mask).clip(0, 255).astype(np.uint8)
    cv2.imwrite(result_path, result)
    print("Result :", result_path)
    print("Completed : ", count , "/", len(img_info))
    count = count + 1




Streaming output truncated to the last 5000 lines.
Result : /content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/face_masked/51--Dresses/51_Dresses_wearingdress_51_620.jpg
Completed :  10381 / 12880
Result : /content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/face_masked/51--Dresses/51_Dresses_wearingdress_51_325.jpg
Completed :  10382 / 12880
Result : /content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/face_masked/51--Dresses/51_Dresses_wearingdress_51_72.jpg
Completed :  10383 / 12880
Result : /content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/face_masked/51--Dresses/51_Dresses_wearingdress_51_834.jpg
Completed :  10384 / 12880
Result : /content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/face_masked/51--Dresses/51_Dresses_wearingdress_51_743.jpg
Completed :  10385 / 12880
Result : /content/drive/MyDrive/cv_project_fall_2022/face_mask/data/wi